# MNIST Using convolution

This is a stipped down version of MNIST.  We will learn how to recognize a 6!

For this demo to work you'll need to download the mnist dataset from [here](http://yann.lecun.com/exdb/mnist/)
and place them in `notebooks/data/`

In [ ]:
## Helper function to read the MNIST data set

import os
import struct
import numpy as np

"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""

def read(dataset = 'training', path = './data', seperator = '-'):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is 'training':
        fname_img = os.path.join(path, 'train-images'+seperator+'idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels'+seperator+'idx1-ubyte')
    elif dataset is 'testing':
        fname_img = os.path.join(path, 't10k-images'+seperator+'idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels'+seperator+'idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        tmp = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)
        tmp2 = np.array(tmp).astype('float')
        img = [ x / 255 for x in tmp2 ]
        

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def show(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

We are going to train and get really good at recognizing this 6

In [ ]:
mnist = read('training','..\\data','.')
#mnist = read('training')
mnist = list(mnist)
label, pixels = mnist[32] ## random 6 i found
show(pixels)

In [ ]:
def onehot(label,nb_labels=10):
    out = np.zeros(nb_labels)
    out[label-1]=1
    return out

In [ ]:
samples = 15
pics = [x[1]for x in mnist[:samples]]
labels = [onehot(x[0]) for x in mnist[:samples]]

In [ ]:
#openmined imports:
import syft
import syft.nn as nn
import syft.controller
import imp
imp.reload(syft.controller)
imp.reload(syft.nn)
imp.reload(syft)
from syft import FloatTensor

### Our data

In [ ]:
data = FloatTensor(pics,autograd=True)
target = FloatTensor(labels).autograd(True)
pic_size = data.shape()[1:3]
data.shape(),target.shape(),pic_size

### Our model

In [ ]:
samples

In [ ]:
def tester(*out_dims,id=None):
    x = list( out_dims)
    print(x,type(x))
    y = [1,2,3]
    print(y,type(y))
tester(1,2,3)

In [ ]:
model = nn.Sequential()
model.add(nn.View(samples,1,pic_size[0],pic_size[1]))
model.add(nn.Conv2d(1,1,7))
model.add(nn.Sigmoid())
model.add(nn.Conv2d(1,1,7))
model.add(nn.Sigmoid())
model.add(nn.View(samples,16*16))
model.add(nn.Linear(16*16,25))
model.add(nn.Sigmoid())
model.add(nn.Linear(5*5,10))
model.add(nn.Sigmoid())

In [ ]:
#pre-training
pred = model(data)
pred.shape(),target.shape()

Train our model to become a pro at that digits

In [ ]:
#%%timeit #-n 3

pred = model(data)
loss = (pred - target) ** 2 # Mean Squared Error Loss

loss.backward()

for i,p in enumerate(model.parameters()):
    if p.autograd():
        p -= p.grad()

print(loss.to_numpy().sum())

In [ ]:
example = 1
pred.to_numpy()[example],target.to_numpy()[example]

Now, let's find a different 6 in the data and see how well we can recognize that

In [ ]:
other_label, other_pixels = mnist[36] ## a different random 6 i found
show(other_pixels)
other_six = FloatTensor(other_pixels).view(1, 28,28).autograd(True)

In [ ]:
#this doesn't work yet, need batch support
pred = model(other_six)
pred

Check out the 6th element of the array!

## Simple examples

In [ ]:
#openmined imports:
import syft
import syft.nn as nn
import syft.controller
import imp
imp.reload(syft.controller)
imp.reload(syft.nn)
imp.reload(syft)
from syft import FloatTensor

In [ ]:
data = FloatTensor( [[[[1,0],[0,1]]]],autograd=1)
target = FloatTensor( [[[[2,1]]]],autograd=1)
model = nn.Conv2d(1,1,(2,1))
model(data),target

In [ ]:
data = FloatTensor( [[[[1,0,1],[0,0,0],[0,1,0]]]],autograd=1)
target = FloatTensor( [[[[1,2],[1,0]]]],autograd=1)
model = nn.Conv2d(1,1,2)
model(data),target

In [ ]:
model.parameters()

In [ ]:
#grad = FloatTensor(-loss.to_numpy(),autograd=1)

loss#.to_numpy()


In [ ]:
loss

In [ ]:
pred = model(data)
print(pred.shape(),target.shape())
loss = (pred - target)# ** 2 # Mean Squared Error Loss

In [ ]:
loss.id,pred.id

In [ ]:
data.grad()

In [ ]:
pred.autograd(),loss.autograd()

In [ ]:
loss.to_numpy()
loss.backward()
loss


In [ ]:
pred = model(data)
print(pred.shape(),target.shape())
loss = (pred - target)# ** 2 # Mean Squared Error Loss
grad = FloatTensor(-loss.to_numpy(),autograd=1)
loss.backward(grad)
loss.grad()

In [ ]:
grad.id,loss.id,grad

In [ ]:
model.parameters()

In [ ]:
pred = model(data)
print(pred.shape(),target.shape())
loss = (pred - target)# ** 2 # Mean Squared Error Loss
loss1 = FloatTensor(-loss.to_numpy())
loss.backward(loss1)
loss.grad(),model.parameters()

In [ ]:
pred = model(data)
print(pred.shape(),target.shape())
loss = (pred - target)# ** 2 # Mean Squared Error Loss
loss.backward()
loss.grad()

In [ ]:
loss,model.parameters()[0],model.parameters()[0].grad()

In [ ]:
loss.grad()
#model.parameters()

In [ ]:
#above, I would like to see the backpropped error, not a backprop of ones

In [ ]:
%%timeit -n 3

pred = model(data)
loss = (pred - target) ** 2 # Mean Squared Error Loss

loss.backward()

for i,p in enumerate(model.parameters()):
    if p.autograd():
        print( p.grad().to_numpy())
        p -= p.grad()*0.01

#print(loss.to_numpy().sum())
print(loss.to_numpy())